In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import astropy.units as u
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord
from gPhoton.pipeline import execute_pipeline

Noebook summary: More work on generating merged catalog.

Pipeline steps:
1. ~Run photometry with extraction on NUV~
2. ~Run photometry with extraction on FUV~
3. ~Run photometry on FUV with NUV source positions~
4. ~Run photometry on NUV with FUV source positions~
5. ~Generate a merged catalog from NUV extractions~
6. ~Generate a merged catalog from FUV extractions~
7. Generate a merged catalog from all NUV & FUV detections

In [2]:
from glcat_merge_utils import *
from lightcurve_utils import *
from glcat_catalog_utils import *

In [3]:
data = accumulate_run_data(23456)
cat_nuv = make_catalog(data,'NUV')
cat_fuv = make_catalog(data,'FUV')

In [4]:
def crossmatch_catalogs(master_table, match_table,
                        match_radius=2.5, # arcseconds
                       ):
    master_catalog = SkyCoord(ra=master_table['RA'].values*u.degree,
                              dec=master_table['DEC'].values*u.degree)
    match_catalog = SkyCoord(ra=match_table['RA'].values*u.degree,
                             dec=match_table['DEC'].values*u.degree)
    catalog_ix, d2d, d3d = match_catalog.match_to_catalog_sky(master_catalog)
    sep_constraint = d2d < match_radius * u.arcsec

    return catalog_ix,d2d

In [5]:
# MATCH FUV TO NUV

idx,d2d=crossmatch_catalogs(cat_nuv,cat_fuv)
match = pd.DataFrame({'idx':idx,'d2d':d2d,
                      'good_match':np.array(d2d <= 2.5 * u.arcsec,dtype=int),
                      'best_match':np.zeros(len(idx),dtype=int),
                      'n_matches':np.zeros(len(idx),dtype=int),
                      'n_good_matches':np.zeros(len(idx),dtype=int)})
for ix in np.unique(idx):
    best_ix = np.argmin(match.loc[match['idx']==ix]['d2d'].values)
    match.loc[match.loc[match['idx']==ix].iloc[best_ix].name,'best_match']=1
    match.loc[match['idx']==ix,'n_matches']=len(match.loc[match['idx']==ix])
    match.loc[match['idx']==ix,'n_good_matches']=len(match.loc[match['idx']==ix].loc[match['good_match']==1])

# update the FUV catalog with the matches to the NUV catalog
cat_fuv['F2N_NUV_MATCH_INDEX']=np.full(len(cat_fuv),-1)
cat_fuv['F2N_MATCH_DISTANCE']=np.full(len(cat_fuv),np.nan)
cat_fuv['F2N_GOOD_MATCH_FLAG']=np.full(len(cat_fuv),-1)
cat_fuv['F2N_BEST_MATCH_FLAG']=np.full(len(cat_fuv),-1)

cat_fuv.loc[match.index,'F2N_NUV_MATCH_INDEX']=np.array(match['idx'].values,dtype=int)
cat_fuv.loc[match.index,'F2N_MATCH_DISTANCE']=match['d2d'].values
cat_fuv.loc[match.index,'F2N_GOOD_MATCH_FLAG']=np.array(match['good_match'].values,dtype=int)
cat_fuv.loc[match.index,'F2N_BEST_MATCH_FLAG']=np.array(match['best_match'].values,dtype=int)

# update the NUV catalog with the reverse lookup to the "best" matches in the FUV catalog
cat_nuv['F2N_FUV_BEST_MATCH_INDEX']=np.full(len(cat_nuv),-1)
cat_nuv['F2N_FUV_BEST_MATCH_DISTANCE']=np.full(len(cat_nuv),np.nan)
cat_nuv['F2N_FUV_GOOD_MATCH_COUNT']=np.full(len(cat_nuv),-1)
#cat_nuv['F2N_BEST_MATCH']=np.full(len(cat_nuv),-1)

# the index in the FUV catalog of the nearest match aka "best match"
cat_nuv.loc[match['idx'][match['best_match']==1],
            'F2N_FUV_BEST_MATCH_INDEX']=np.array(
                match.loc[match['best_match']==1]['idx'].values,dtype=int)
# this distance in decimal degress (?) to the nearest match aka "best match"
cat_nuv.loc[match['idx'][match['best_match']==1],
            'F2N_FUV_BEST_MATCH_DISTANCE']=np.array(
                match.loc[match['best_match']==1]['d2d'].values)
# it is possible that the "best match" is not a "good match" in which case this will be zero
cat_nuv.loc[match['idx'][match['best_match']==1],
            'F2N_FUV_GOOD_MATCH_COUNT']=np.array(
                match.loc[match['best_match']==1]['n_good_matches'].values)

In [6]:
# MATCH NUV TO FUV

idx,d2d=crossmatch_catalogs(cat_fuv,cat_nuv)
match = pd.DataFrame({'idx':idx,'d2d':d2d,
                      'good_match':np.array(d2d <= 2.5 * u.arcsec,dtype=int),
                      'best_match':np.zeros(len(idx),dtype=int),
                      'n_matches':np.zeros(len(idx),dtype=int),
                      'n_good_matches':np.zeros(len(idx),dtype=int)})
for ix in np.unique(idx):
    best_ix = np.argmin(match.loc[match['idx']==ix]['d2d'].values)
    match.loc[match.loc[match['idx']==ix].iloc[best_ix].name,'best_match']=1
    match.loc[match['idx']==ix,'n_matches']=len(match.loc[match['idx']==ix])
    match.loc[match['idx']==ix,'n_good_matches']=len(match.loc[match['idx']==ix].loc[match['good_match']==1])

# update the NUV catalog with the matches to the FUV catalog
cat_nuv['N2F_NUV_MATCH_INDEX']=np.full(len(cat_nuv),-1)
cat_nuv['N2F_MATCH_DISTANCE']=np.full(len(cat_nuv),np.nan)
cat_nuv['N2F_GOOD_MATCH_FLAG']=np.full(len(cat_nuv),-1)
cat_nuv['N2F_BEST_MATCH_FLAG']=np.full(len(cat_nuv),-1)

cat_nuv.loc[match.index,'N2F_NUV_MATCH_INDEX']=np.array(match['idx'].values,dtype=int)
cat_nuv.loc[match.index,'N2F_MATCH_DISTANCE']=match['d2d'].values
cat_nuv.loc[match.index,'N2F_GOOD_MATCH_FLAG']=np.array(match['good_match'].values,dtype=int)
cat_nuv.loc[match.index,'N2F_BEST_MATCH_FLAG']=np.array(match['best_match'].values,dtype=int)

# update the FUV catalog with the reverse lookup to the "best" matches in the NUV catalog
cat_fuv['N2F_FUV_BEST_MATCH_INDEX']=np.full(len(cat_fuv),-1)
cat_fuv['N2F_FUV_BEST_MATCH_DISTANCE']=np.full(len(cat_fuv),np.nan)
cat_fuv['N2F_FUV_GOOD_MATCH_COUNT']=np.full(len(cat_fuv),-1)
#cat_nuv['F2N_BEST_MATCH']=np.full(len(cat_nuv),-1)

# the index in the NUV catalog of the nearest match aka "best match"
cat_fuv.loc[match['idx'][match['best_match']==1],
            'N2F_FUV_BEST_MATCH_INDEX']=np.array(
                match.loc[match['best_match']==1]['idx'].values,dtype=int)
# this distance in decimal degress (?) to the nearest match aka "best match"
cat_fuv.loc[match['idx'][match['best_match']==1],
            'N2F_FUV_BEST_MATCH_DISTANCE']=np.array(
                match.loc[match['best_match']==1]['d2d'].values)
# it is possible that the "best match" is not a "good match" in which case this will be zero
cat_fuv.loc[match['idx'][match['best_match']==1],
            'N2F_FUV_GOOD_MATCH_COUNT']=np.array(
                match.loc[match['best_match']==1]['n_good_matches'].values)

In [7]:
cat_nuv

,ECLIPSE,RA,DEC,NUV_XCENTER,NUV_YCENTER,NUV_SUM_APER0,NUV_EDGE_APER0,NUV_MASK_APER0,NUV_CPS_APER0,NUV_CPS_ERR_APER0,...,FUV_MASK_FLAG_APER6,FUV_EDGE_FLAG_APER6,FUV_EXPT,F2N_FUV_BEST_MATCH_INDEX,F2N_FUV_BEST_MATCH_DISTANCE,F2N_FUV_GOOD_MATCH_COUNT,N2F_NUV_MATCH_INDEX,N2F_MATCH_DISTANCE,N2F_GOOD_MATCH_FLAG,N2F_BEST_MATCH_FLAG
0,23456,323.448811,-2.638534,1638.452026,170.996174,74.195186,1,0,0.049456,0.005742,...,0,1,1678.19451,-1,NaN,-1,1,0.014035,0,0
1,23456,323.458572,-2.636944,1615.050994,174.814059,35.807291,0,0,0.023868,0.003989,...,0,1,1678.19451,-1,NaN,-1,1,0.010425,0,0
2,23456,323.448619,-2.635363,1638.913971,178.608991,114.462236,0,0,0.076297,0.007131,...,0,1,1678.19451,-1,NaN,-1,1,0.011628,0,0
3,23456,323.509375,-2.632623,1493.247071,185.185578,50.258974,0,0,0.033501,0.004726,...,0,0,1678.19451,3,0.000732,0,0,0.000732,0,1
4,23456,323.459455,-2.632761,1612.932867,184.853847,33.605723,0,0,0.022400,0.003864,...,0,0,1678.19451,4,0.006737,0,1,0.006737,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4835,23456,323.546199,-1.505455,1404.865650,2890.475707,63.903954,0,0,0.042596,0.005329,...,0,0,1678.19451,-1,NaN,-1,3453,0.008293,0,0
4836,23456,323.548553,-1.504981,1399.217901,2891.612115,80.647389,0,1,0.053757,0.005986,...,0,0,1678.19451,-1,NaN,-1,3453,0.005895,0,0
4837,23456,323.543522,-1.505130,1411.288694,2891.256176,47.527211,0,0,0.031680,0.004595,...,0,0,1678.19451,-1,NaN,-1,3454,0.010413,0,0
4838,23456,323.541941,-1.501864,1415.081510,2899.095427,51.393869,0,1,0.034257,0.004779,...,0,0,1678.19451,-1,NaN,-1,3454,0.009242,0,0


In [8]:
cat_fuv

,ECLIPSE,RA,DEC,FUV_XCENTER,FUV_YCENTER,FUV_SUM_APER0,FUV_EDGE_APER0,FUV_MASK_APER0,FUV_CPS_APER0,FUV_CPS_ERR_APER0,...,NUV_MASK_FLAG_APER6,NUV_EDGE_FLAG_APER6,NUV_EXPT,F2N_NUV_MATCH_INDEX,F2N_MATCH_DISTANCE,F2N_GOOD_MATCH_FLAG,F2N_BEST_MATCH_FLAG,N2F_FUV_BEST_MATCH_INDEX,N2F_FUV_BEST_MATCH_DISTANCE,N2F_FUV_GOOD_MATCH_COUNT
0,23456,323.509837,-2.632055,1460.456727,196.750320,20.305039,0,0,0.012099,0.002685,...,0,0,1500.228677,3,0.000732,0,1,0,0.000732,0
1,23456,323.456429,-2.626741,1588.506153,209.505800,12.559613,0,0,0.007484,0.002112,...,0,0,1500.228677,4,0.006737,0,1,1,0.006737,0
2,23456,323.480038,-2.621575,1531.901204,221.905105,7.915088,0,0,0.004716,0.001676,...,0,0,1500.228677,10,0.001323,0,1,2,0.001323,0
3,23456,323.589603,-2.621367,1269.209061,222.391674,10.358416,0,0,0.006172,0.001918,...,0,0,1500.228677,14,0.000414,1,1,3,0.000414,1
4,23456,323.434578,-2.615019,1640.896670,237.639575,94.887268,0,0,0.056541,0.005804,...,0,0,1500.228677,13,0.002493,0,1,4,0.002493,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3957,23456,323.746538,-1.551791,892.500563,2789.420749,2.219125,0,0,0.001322,0.000888,...,0,1,1500.228677,4826,0.014218,0,0,-1,NaN,-1
3958,23456,323.332308,-1.538430,1886.335470,2821.518723,44.113045,0,1,0.026286,0.003958,...,0,0,1500.228677,4590,0.001364,0,0,3958,0.005033,0
3959,23456,323.330821,-1.536653,1889.905329,2825.784542,25.787951,0,0,0.015366,0.003026,...,0,0,1500.228677,4590,0.000988,0,1,3959,0.000988,0
3960,23456,323.604938,-1.515287,1232.225045,2877.069324,7.935622,0,0,0.004729,0.001679,...,0,0,1500.228677,4627,0.003126,0,0,-1,NaN,-1


In [9]:
cat_nuv.to_csv('/Users/cm/Downloads/nuvcat_240612.csv')
cat_fuv.to_csv('/Users/cm/Downloads/fuvcat_240612.csv')